In [1]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

## 上下文压缩
检索的一个挑战是，当您将数据摄取到系统中时，您通常不知道文档存储系统将面临的特定查询。这意味着与查询最相关的信息可能隐藏在包含大量不相关文本的文档中。在应用程序中传递完整的文档可能会导致LLM调用成本更高，响应也更差。
上下文压缩就是为了解决这个问题。其思想很简单:与其按原样立即返回检索到的文档，不如使用给定查询的上下文压缩它们，以便只返回相关信息。这里的“压缩”既指压缩单个文档的内容，也指过滤掉整个文档。
要使用上下文压缩检索器，您需要:
- 基本检索器
- 文档压缩器

上下文压缩检索器将查询传递给基本检索器，获取初始文档并通过文档压缩器传递它们。Document Compressor获取一个文档列表，并通过减少文档的内容或完全删除文档来缩短它。

In [2]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 使用一个寻常的RAG
让我们首先初始化一个简单的向量存储检索器，并存储文档(以块为单位)。我们可以看到，给定一个示例问题，我们的检索器返回一两个相关的文档和一些不相关的文档。即使是相关的文档也有很多不相关的信息。

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

documents = TextLoader("../../data/2015北京年报.txt",encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.get_relevant_documents(
    "增值收益及其收益率为多少?"
)
pretty_print_docs(docs)

Created a chunk of size 720, which is longer than the specified 500
Created a chunk of size 818, which is longer than the specified 500
Created a chunk of size 727, which is longer than the specified 500


Document 1:

（二）业务支出
2015全年,住房公积金业务支出共计300088.48万元，同比降低45.2%。其中，缴存职工账户余额的利息支出251072.41万元，归集手续费用支出676.66万元，委托贷款手续费支出45299.35万元，其他支出3040.06万元（主要是项目贷款应交税金）。同时，经市财政局批复后支出与住房公积金业务相关的专项经费32993.82万元（主要为公积金贷款资产委托管理费、公积金个人住房贷款贴息经费、综合信息管理系统升级改造、运维服务等业务支出费用）。

（三）增值收益
管理中心在利率连续下调的背景下，通过加强资金综合调度，提升资金运用率，优先安排个贷资金发放，减少银行存款，住房公积金增值收益大幅提高。2015全年，住房公积金增值收益788646.16万元，创历史新高，同比增长65.8%。增值收益率（增值收益与月均缴存余额的比率）2.9%，比上年同期增加 0.9个百分点。
----------------------------------------------------------------------------------------------------
Document 2:

（二）提取业务
2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。
----------------------------------------------------------------------------------------------------
Document 3:

（四）购买国债
2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。

（五）资金存储
截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。

（六）资金运用率

## 使用`LLMChainExtractor`添加上下文压缩
现在让我们用一个`ContextualCompressionRetriever`来包装我们的基本检索器。我们将添加一个`LLMChainExtractor`，它将遍历最初返回的文档，并仅从每个文档中提取与查询相关的内容。

In [4]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "增值收益及其收益率为多少?"
)
pretty_print_docs(compressed_docs)

E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

住房公积金增值收益788646.16万元，创历史新高，同比增长65.8%。增值收益率（增值收益与月均缴存余额的比率）2.9%，比上年同期增加 0.9个百分点。
----------------------------------------------------------------------------------------------------
Document 2:

2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。
----------------------------------------------------------------------------------------------------
Document 3:

存款(含增值收益存款)利息收入157379.86万元，委托贷款利息收入930128.01万元，国债利息收入517.15万元
----------------------------------------------------------------------------------------------------
Document 4:

2015年住房公积金实缴人数和缴存额增长率分别为14.0%和13.8%。


## 更多内置压缩器:过滤器
### LLMChainFilter
`LLMChainFilter`是一个稍微简单但更健壮的压缩器，它使用LLM链来决定过滤掉哪些最初检索到的文档以及返回哪些文档，而不需要操作文档内容。

In [5]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "增值收益及其收益率为多少?"
)
pretty_print_docs(compressed_docs)

E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
E:\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

（二）业务支出
2015全年,住房公积金业务支出共计300088.48万元，同比降低45.2%。其中，缴存职工账户余额的利息支出251072.41万元，归集手续费用支出676.66万元，委托贷款手续费支出45299.35万元，其他支出3040.06万元（主要是项目贷款应交税金）。同时，经市财政局批复后支出与住房公积金业务相关的专项经费32993.82万元（主要为公积金贷款资产委托管理费、公积金个人住房贷款贴息经费、综合信息管理系统升级改造、运维服务等业务支出费用）。

（三）增值收益
管理中心在利率连续下调的背景下，通过加强资金综合调度，提升资金运用率，优先安排个贷资金发放，减少银行存款，住房公积金增值收益大幅提高。2015全年，住房公积金增值收益788646.16万元，创历史新高，同比增长65.8%。增值收益率（增值收益与月均缴存余额的比率）2.9%，比上年同期增加 0.9个百分点。
----------------------------------------------------------------------------------------------------
Document 2:

（二）提取业务
2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。
----------------------------------------------------------------------------------------------------
Document 3:

（四）购买国债
2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。

（五）资金存储
截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。

（六）资金运用率

### EmbeddingsFilter
对每个检索到的文档进行额外的LLM调用既昂贵又缓慢。EmbeddingsFilter通过嵌入文档和查询并只返回那些与查询具有足够相似嵌入的文档，提供了一种更便宜和更快的选择。

In [6]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "增值收益及其收益率为多少?"
)
pretty_print_docs(compressed_docs)

Document 1:

（二）业务支出
2015全年,住房公积金业务支出共计300088.48万元，同比降低45.2%。其中，缴存职工账户余额的利息支出251072.41万元，归集手续费用支出676.66万元，委托贷款手续费支出45299.35万元，其他支出3040.06万元（主要是项目贷款应交税金）。同时，经市财政局批复后支出与住房公积金业务相关的专项经费32993.82万元（主要为公积金贷款资产委托管理费、公积金个人住房贷款贴息经费、综合信息管理系统升级改造、运维服务等业务支出费用）。

（三）增值收益
管理中心在利率连续下调的背景下，通过加强资金综合调度，提升资金运用率，优先安排个贷资金发放，减少银行存款，住房公积金增值收益大幅提高。2015全年，住房公积金增值收益788646.16万元，创历史新高，同比增长65.8%。增值收益率（增值收益与月均缴存余额的比率）2.9%，比上年同期增加 0.9个百分点。
----------------------------------------------------------------------------------------------------
Document 2:

（二）提取业务
2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。
----------------------------------------------------------------------------------------------------
Document 3:

（四）购买国债
2015年，未发生新购买、兑付、转让、收回国债情况。期末国债余额4.26亿元，与上年相比无变化。

（五）资金存储
截至2015年底，管理中心结余资金存款348.31亿元，其中，活期4.02亿元，1年以内定期（含）121.15亿元，1年以上定期172.79亿元，其他(协议、协定、通知存款等)50.35亿元。

（六）资金运用率

## 将上下文压缩和文件转换器串在一起
使用`DocumentCompressorPipeline`，我们还可以很容易地按顺序组合多个压缩器。除了压缩器，我们还可以在管道中添加`BaseDocumentTransformer`，它不执行任何上下文压缩，而只是对一组文档执行一些转换。例如，`TextSplitter`可以用作文档转换器，将文档分成更小的部分，`EmbeddingsRedundantFilter`可以用于根据文档之间的嵌入相似性过滤掉冗余文档。

下面我们创建一个压缩器管道，首先将我们的文档分成更小的块，然后删除冗余的文档，然后根据与查询的相关性进行过滤。

In [7]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator="\n")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [8]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "增值收益及其收益率为多少?"
)
pretty_print_docs(compressed_docs)

Document 1:

管理中心在利率连续下调的背景下，通过加强资金综合调度，提升资金运用率，优先安排个贷资金发放，减少银行存款，住房公积金增值收益大幅提高。2015全年，住房公积金增值收益788646.16万元，创历史新高，同比增长65.8%。增值收益率（增值收益与月均缴存余额的比率）2.9%，比上年同期增加 0.9个百分点。
----------------------------------------------------------------------------------------------------
Document 2:

（一）业务收入
2015全年,住房公积金业务收入共计1088734.64万元，同比增长6.4%。其中，存款(含增值收益存款)利息收入157379.86万元，委托贷款利息收入930128.01万元，国债利息收入517.15万元，其他收入709.62万元。
----------------------------------------------------------------------------------------------------
Document 3:

按收入水平，低收入群体（住房公积金缴存基数低于2014年本市职工平均工资）占53.1%，中等收入群体（住房公积金缴存基数介于2014年本市职工平均工资1倍-3倍之间）占38.5 %，高收入群体（住房公积金缴存基数高于2014年本市职工平均工资3倍）占8.4%。
----------------------------------------------------------------------------------------------------
Document 4:

（二）提取业务
2015全年提取住房公积金1117.38万笔、920.62亿元。提取的金额中，住房消费提取占87.8%（购买、建造、翻建、大修自住住房占77.7 %，偿还购房贷款本息占8.4%，租赁住房占1.7%）；非住房消费提取占12.2%（离休和退休提取占9.8%，完全丧失劳动能力并与单位终止劳动关系提取占0.2%，户口迁出本市或出境定居占0.3%,其他占1.9%）。
---------------------------------------------